## 1_fetch_OSM_data_from_DB2
### Labelled OSM buildings reside in DB2 already as those were merged to the VIDA data set. This notebook fetches them from the database serving as a basis for the labelled data set

### Initial configuration
#### To start working with this particular notebook, you need to provide necessary credential and settings
#### Below is an template of configuration, which is necessary prepare aside of this notebook and copy & paste all content in triple quotes to the next cell's input field
    """
    {
    "COS_ENDPOINT_URL": "s3.private.eu-de.cloud-object-storage.appdomain.cloud",
    "COS_AUTH_ENDPOINT_URL": "https://iam.cloud.ibm.com/oidc/token",
    "COS_APIKEY": "xxx",
    "DATA_CURATION_BUCKET": "xxx",
    "DB2_CONNECTION_STRING": "jdbc:db2://65beb513-5d3d-4101-9001-f42e9dc954b3.brt9d04f0cmqeb8u7740.databases.appdomain.cloud:30371/BLUDB:sslConnection=true;useJDBC4ColumnNameAndLabelSemantics=false;db2.jcc.charsetDecoderEncoder=3;",
    "DB2_USERNAME": "xxx",
    "DB2_PASSWORD": "xxx",
    "COUNTRY_TABLE": "FEATURES_DB_VIDA_EXTENDED",
    "AREA_THRESHOLD": 20
    }
    """


In [ ]:
# Read notebook configuration
import getpass
import json

config_str = getpass.getpass('Enter your prepared config: ')
config = json.loads(config_str)

In [ ]:
# Import necessary libraries
import geopandas as gpd
import pandas as pd
from pyproj import Geod
import shapely
import jaydebeapi as jdbc
import jpype
import os
from botocore.client import Config
import ibm_boto3

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

geod = Geod(ellps="WGS84")


In [ ]:
osm_labelled_parquet = "ML_OSM_dataset.parquet"
curation_bucket = config["DATA_CURATION_BUCKET"]

In [ ]:
# init S3 client in order to upload data to the curation bucket
cos_client = ibm_boto3.client(service_name='s3',
                              ibm_api_key_id=config["COS_APIKEY"],
                              ibm_auth_endpoint=config["COS_AUTH_ENDPOINT_URL"],
                              config=Config(signature_version='oauth'),
                              endpoint_url=config["COS_ENDPOINT_URL"])

In [ ]:
# connect to the IBM DB2 function
def connect_to_db():

    jar = 'db2jcc4.jar'
    os.environ['CLASSPATH'] = jar

    args='-Djava.class.path=%s' % jar
    jvm_path = jpype.getDefaultJVMPath()
    try:
        jpype.startJVM(jvm_path, args)
    except Exception as e:
        print('startJVM exception: ', e)
        
    if jpype.isJVMStarted() and not jpype.isThreadAttachedToJVM():
        jpype.attachThreadToJVM()
        jpype.java.lang.Thread.currentThread().setContextClassLoader(jpype.java.lang.ClassLoader.getSystemClassLoader())
        
    # create JDBC connection
    conn = jdbc.connect(
                'com.ibm.db2.jcc.DB2Driver',
                config['DB2_CONNECTION_STRING'],
                [config["DB2_USERNAME"], config["DB2_PASSWORD"]],
                'db2jcc4.jar')
    
    return conn


def get_osm_entries():
    '''
        Fetch all OSM buildings from desired DB2 table
    '''
    columns = ['id', 'latitude', 'longitude', 'polygon_coordinates', 'vida_confidence', 'osm_type']
    
    # sql statement for selecting entries by defined rectangle boundaries
    sql = f"""
        SELECT {', '.join(columns)} FROM USER1.{config["COUNTRY_TABLE"]}
        WHERE 
            (AREA_IN_METERS > {config["AREA_THRESHOLD"]}) AND
            (FOOTPRINT_SOURCE = 'osm')
        """
    
    try:
        cursor.execute(sql)
        data = cursor.fetchall()
    except Exception as e:
        print(f"Fetch items error occured: {e}")
        print("Reconnecting to the database try again...")

        conn = connect_to_db()
        cursor = conn.cursor()
        cursor.execute(sql)
        data = cursor.fetchall()
    finally:
        # reshape obtained data to the GeoDataFrame
        df = pd.DataFrame(data=data, columns=columns)
        df = gpd.GeoDataFrame(df, geometry=shapely.from_wkt(df.polygon_coordinates.astype(str)))
        df = df.drop(['polygon_coordinates'], axis=1)
        df['building_area_in_meters'] = df["geometry"].apply(lambda g: abs(geod.geometry_area_perimeter(g)[0]))

        return df
    

In [ ]:
# fetch & save buildings to the parquet file
ML_OSM_df = get_osm_entries()
ML_OSM_df.to_parquet(osm_labelled_parquet)

# optionaly upload file to the bucket
if type(curation_bucket) == str:
        
    try:
        cos_client.upload_file(
            Filename=osm_labelled_parquet,
            Bucket=curation_bucket,
            Key=osm_labelled_parquet,
            ExtraArgs={'ContentDisposition': 'attachment'}
        )
           
        print(f'File {osm_labelled_parquet} successfully uploaded to the COS {curation_bucket} bucket')
    except Exception as e:
        print(f"\033[91mFailed upload file to the bucket {curation_bucket}. Error: {e}")